In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv

# FIT01 for Fuel Cycle Simulation Reactor Depletion
### This Functionality Isolation Test (FIT) is to test MOX fuel fabrication and depletion functionalities in a reactor module given streams of varying plutonium quality (different isotopics). The four streams are:
### `ref, source_1, source_2, and source_3`.

### ORION is used for this analysis, where a MOX LWR fabrication and reactor module is given four streams with varying plutonium stream. 

### The Fuel Loading Module (FLM) is the `Fuel Fabrication` facility in ORION, that uses cross sections and nu-bars to calculate how to mix the fissile and fertile stream. The datasets with `_ff` are ones using fixed fractions, which means that the FLM is not utilized. (DOESN'T WORK)


### The fissile stream is plutonium and americium in different qualities (higher the source number, lower the quality - less fissile Pu isotopes).
### The fertile stream is depleted uranium (0.238% U235).

### Power density parameter in ORION is calculated by dividng the thermal power by the core mass
```
lwr_mox = 3000MW / 72t = 41.6W/g
sfr = 2500 MW / 51.95t = 48.12 W/g
```

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
def get_comp_dict(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        isotopes = lines[0]
        isotopes = isotopes.split(',')[1:-1]
        isotopes = [x.split('+')[1] for x in isotopes]
        masses = lines[-1]
        masses = masses.split(',')[1:-1]
        masses = np.array([float(x) for x in masses])
        total_mass = sum(masses)
    
    comp_dict = {}
    for indx, val in enumerate(isotopes):
        comp_dict[val] = masses[indx]
    
    # get rid of oxygen
    comp_dict = {key: value for key, value in comp_dict.items()
                 if 'O' not in key}
    
    # from mass to percentage
    comp_dict = {key: value / sum(comp_dict.values()) * 100 for key, value
                 in comp_dict.items()}

    return comp_dict

In [4]:
def show_fracs(comp_dict, search_list=[]):
    filtered_dict = {}
    for key, val in comp_dict.items():
        if val < 1e-5:
            continue
        if len(search_list) > 1:
            for keyword in search_list:
                if keyword.upper() in key:
                    filtered_dict[key] = val
        else:
            filtered_dict[key] = val
    return filtered_dict

In [5]:
def get_am_pu(filtered_dict):
    am_pu = 0
    for key, val in filtered_dict.items():
        if 'AM' in key or 'PU' in key:
            am_pu += val
    return am_pu

In [6]:
def main(file_list):
    data = {}
    for file in file_list:
        comp_dict = get_comp_dict(file)
#        frac_dict = show_fracs(comp_dict, ['U', 'NP', 'PU', 'AM'])
        frac_dict = show_fracs(comp_dict,['U', 'PU', 'NP', 'AM'])
        am_pu = get_am_pu(frac_dict)
        filename = file.split('/')[-1]
        filename = filename.split('.csv')[0]
        data.update({filename:{'fraction': frac_dict, 'AM+PU': am_pu}})
    return data

In [7]:
from os import listdir
from os.path import isfile, join
file_list = [f for f in listdir('./results') if isfile(join('./results', f))]
file_list =['./results/' + x for x in file_list]

In [8]:
data = main(file_list)

In [9]:
# show all 
def show_data(data, must_word=''):
    for key, val in data.items():
        if must_word not in key:
            continue
        print('=========================')
        print(key)
        for key2, val2 in val.items():
            if key2 == 'fraction':
                for key3, val3 in val2.items():
                    print('%s = %f' %(key3, val3))
            else:
                print('%s = %f' %(key2, val2))
        print('=========================')
        print('\n')


In [10]:
import pandas as pd 

def make_csv(data, file_list, iso_list, outputfile):
    result = np.vstack(iso_list + ['Am+Pu'])
    for file in file_list:
        array = np.zeros(len(iso_list) + 1)
        for key2, val2 in data[file]['fraction'].items():
            if key2 in iso_list:
                indx = iso_list.index(key2)
                array[indx] = val2
        array[-1] = data[file]['AM+PU']
        result = np.hstack((result, np.vstack(array)))
    df = pd.DataFrame(result)
    df.to_csv(outputfile)
        
"""
        if key in file_list:
            array = np.zeros(len(iso_list) + 1)
            for key2, val2 in  val['fraction'].items():
                if key2 in iso_list:
                    indx = iso_list.index(key2)
                    array[indx] = val2
            array[-1] = val['AM+PU']
            result = np.hstack((result, np.vstack(array)))
    df = pd.DataFrame(result)
    df.to_csv(outputfile)
"""

"\n        if key in file_list:\n            array = np.zeros(len(iso_list) + 1)\n            for key2, val2 in  val['fraction'].items():\n                if key2 in iso_list:\n                    indx = iso_list.index(key2)\n                    array[indx] = val2\n            array[-1] = val['AM+PU']\n            result = np.hstack((result, np.vstack(array)))\n    df = pd.DataFrame(result)\n    df.to_csv(outputfile)\n"

In [11]:

make_csv(data, ['pu_ref_charge', 'source_1_charge', 'source_2_charge', 'source_3_charge'],
         ['U235', 'U238', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'mox_lwr_charge.csv')

make_csv(data, ['pu_ref_discharge', 'source_1_discharge', 'source_2_discharge', 'source_3_discharge'],
         ['U234', 'U235', 'U236', 'U238', 'Np237', 'PU236', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'mox_lwr_discharge.csv')

make_csv(data, ['sfr_ref_charge', 'sfr_1_charge', 'sfr_2_charge', 'sfr_3_charge'],
         ['U235', 'U238', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'sfr_charge.csv')

make_csv(data, ['sfr_ref_discharge', 'sfr_ref_discharge_blanket', 'sfr_1_discharge', 'sfr_1_discharge_blanket',
                'sfr_2_discharge', 'sfr_2_discharge_blanket', 'sfr_3_discharge', 'sfr_3_discharge_blanket'],
         ['U234', 'U235', 'U236', 'U238', 'Np237', 'PU236', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'sfr_discharge.csv')



In [12]:
def csv_to_latex_table(csvfile, sfr_discharge=False):
    with open(csvfile, 'r') as f:
        lines = f.readlines()
        lines = lines[1:]
        columns = len(lines[0].split(',')) - 1
        string = '\t\\begin{tabular}{' + 'c'*columns + '}\n'
        string += '\t\t\\hline\n'
        if sfr_discharge:
            string += '\t\t\\textbf{wt\%} & \\textbf{ref driver} & \\textbf{ref blanket} & \\textbf{one driver} & \\textbf{one blanket} & \\textbf{two driver} & \\textbf{two blanket} & \\textbf{three driver} & \\textbf{three blanket} \\\ '
        else:
            string += '\t\t\\textbf{wt\%} & \\textbf{ref} & \\textbf{source one} & \\textbf{source two} & \\textbf{source three} \\\ \n'
        string += '\t\t\\hline\n'
        for line in lines:
            line = line.split(',')[1:]
            string += '\t\t'
            for indx, i in enumerate(line):
                if indx == len(line) - 1:
                    string += str(i)[:5].replace('\n','') + ' \\\ \n'
                else:
                    string += str(i)[:5] + ' & '
        string += '\t\t\hline \n'
        string += '\t\end{tabular} \n'
        print(string)
    
    with open(csvfile.replace('.csv','.tex'), 'w') as w:
        w.write(string)

In [13]:
csv_to_latex_table('mox_lwr_charge.csv')
csv_to_latex_table('mox_lwr_discharge.csv')
csv_to_latex_table('sfr_charge.csv')
csv_to_latex_table('sfr_discharge.csv', True)

	\begin{tabular}{ccccc}
		\hline
		\textbf{wt\%} & \textbf{ref} & \textbf{source one} & \textbf{source two} & \textbf{source three} \\ 
		\hline
		U235 & 0.221 & 0.221 & 0.221 & 0.221 \\ 
		U238 & 92.77 & 92.77 & 92.77 & 92.77 \\ 
		PU238 & 0.138 & 0.218 & 0.200 & 0.279 \\ 
		PU239 & 4.357 & 3.611 & 3.289 & 2.697 \\ 
		PU240 & 1.574 & 1.702 & 2.373 & 1.719 \\ 
		PU241 & 0.559 & 0.822 & 0.317 & 1.112 \\ 
		PU242 & 0.349 & 0.562 & 0.764 & 0.894 \\ 
		AM241 & 0.018 & 0.082 & 0.053 & 0.296 \\ 
		Am+Pu & 6.999 & 6.999 & 6.999 & 6.999 \\ 
		\hline 
	\end{tabular} 

	\begin{tabular}{ccccc}
		\hline
		\textbf{wt\%} & \textbf{ref} & \textbf{source one} & \textbf{source two} & \textbf{source three} \\ 
		\hline
		U234 & 0.002 & 0.002 & 0.003 & 0.005 \\ 
		U235 & 0.117 & 0.108 & 0.099 & 0.099 \\ 
		U236 & 0.023 & 0.024 & 0.026 & 0.026 \\ 
		U238 & 90.51 & 89.89 & 89.85 & 89.86 \\ 
		Np237 & 0.0 & 0.0 & 0.0 & 0.0 \\ 
		PU236 & 0.0 & 0.0 & 0.0 & 0.0 \\ 
		PU238 & 0.120 & 0.205 & 0.170 & 0.320 \\ 
	

In [14]:
show_data(data, 'source_3')

source_3_charge_ff
U235 = 0.221340
U238 = 92.778666
PU238 = 0.200900
PU239 = 3.289297
PU240 = 2.373698
PU241 = 0.317800
PU242 = 0.764399
AM241 = 0.053900
AM+PU = 6.999994


source_3_discharge_ff
U234 = 0.004056
U235 = 0.221589
U236 = 0.000658
U238 = 92.779208
NP237 = 0.000307
PU238 = 0.196659
PU239 = 3.289062
PU240 = 2.373037
PU241 = 0.279070
PU242 = 0.764400
AM241 = 0.091954
AM+PU = 6.994181


source_3_charge
U235 = 0.221340
U238 = 92.778666
PU238 = 0.280000
PU239 = 2.697098
PU240 = 1.719198
PU241 = 1.112999
PU242 = 0.894599
AM241 = 0.296100
AM+PU = 6.999994


source_3_discharge
RU103 = 0.007070
RU105 = 0.000028
RU106 = 0.039796
EU154 = 0.007440
EU155 = 0.001924
EU156 = 0.000686
U234 = 0.005245
U235 = 0.099203
U236 = 0.026581
U237 = 0.000288
U238 = 89.869023
U239 = 0.000065
NP237 = 0.019968
NP238 = 0.000049
NP239 = 0.009426
PU238 = 0.320432
PU239 = 1.760305
PU240 = 1.353481
PU241 = 0.863595
PU242 = 0.814439
PU243 = 0.000141
PU244 = 0.000097
AM241 = 0.116600
AM242 = 0.000175
AM242(1) =

## major errors spotted:
1. The charge fuel composition does not change
    * the Pu composition changes, obviously becasue the input stream is different.
    * but the U composition does not, which means that the fuel fabrication is mixing the fuel at the same ratio
        * Maybe it's a cross section thing, so I tried using CASMO mox cross section file, still the same.
    * EFMC doesn't work?
2. The SFR fuel cross sections need to be averaged
    * using the blanket cross sections **overbreed** the Pu+MA
    * using the driver cross sections **deplete** the Pu+MA
    * the average between the two results are similar to the excel results provided